In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

In [9]:
# Load files
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
data = load_doc('data/shakespeare.txt')

#Tokenize
corpus = data.lower().split("\n")
tokenizer = Tokenizer(filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(corpus)
#Find total number of words for the loop
total_words = len(tokenizer.word_index) + 1

In [10]:
input_sequences = []
# Transform text into list of numbers...
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    # Create n-grams
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
# add zeros to the beginnings of sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# Create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
# One-hot encoding
label = ku.to_categorical(label, num_classes=total_words)

In [11]:
# LSTM model
model = Sequential()
# Adding word embeddings here with 100 dimensions
model.add(Embedding(total_words, 300, input_length=max_sequence_len-1))
# Bidirectional LSTM to keep information from both past and future...
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# Final layer, size = number of words in corpus
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 300)           1015800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 300)           541200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 1693)              170993    
_________________________________________________________________
dense_3 (Dense)              (None, 3386)              5735884   
Total params: 7,624,277
Trainable params: 7,624,277
Non-trainable params: 0
____________________________________________

In [12]:
history = model.fit(predictors, label, epochs=100, verbose=1, use_multiprocessing=True)

Train on 15430 samples
Epoch 1/100
15430/15430 [==============================] - 35s 2ms/sample - loss: 6.8996 - accuracy: 0.0209
Epoch 2/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 6.4940 - accuracy: 0.0214
Epoch 3/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 6.3848 - accuracy: 0.0255
Epoch 4/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 6.2644 - accuracy: 0.0312
Epoch 5/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 6.1747 - accuracy: 0.0353
Epoch 6/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 6.1002 - accuracy: 0.0392
Epoch 7/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 6.0226 - accuracy: 0.0406
Epoch 8/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 5.9378 - accuracy: 0.0430
Epoch 9/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 5.8403 - accuracy: 0.0495
Epoch

15430/15430 [==============================] - 32s 2ms/sample - loss: 1.4005 - accuracy: 0.7587
Epoch 77/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.3935 - accuracy: 0.7574
Epoch 78/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.3776 - accuracy: 0.7614
Epoch 79/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.3489 - accuracy: 0.7673
Epoch 80/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.3402 - accuracy: 0.7681
Epoch 81/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.3192 - accuracy: 0.7726
Epoch 82/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.3091 - accuracy: 0.7738
Epoch 83/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.2908 - accuracy: 0.7768
Epoch 84/100
15430/15430 [==============================] - 32s 2ms/sample - loss: 1.2665 - accuracy: 0.7828
Epoch 85/100
15430/15430 [======

In [13]:
model.save('model-embeddings.h5')

In [14]:
# load the model
seed_text = "o know sweet love"
next_words = 50
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

o know sweet love i always write of you be slain lie hid be leave thee look it see her summer heaven work thee half thy parts will thine be of good mind foot lies writ hate live some in heaven sun will look so sun hate blind smell more best sweet tongue you
